In [1]:
import bpy
import os
import io
import contextlib

In [10]:
def apply_texture(obj, texture_path):
    # 오브젝트 모드로 전환
    bpy.ops.object.mode_set(mode='OBJECT')

    # UV 맵 생성
    # bpy.ops.object.mode_set(mode='EDIT')
    # bpy.ops.mesh.select_all(action='SELECT')
    # bpy.ops.uv.smart_project()
    # bpy.ops.object.mode_set(mode='OBJECT')

    # 텍스쳐 이미지 로드
    img = bpy.data.images.load(texture_path)

    # 새 마테리얼 생성
    mat = bpy.data.materials.new(name="CustomMaterial")
    mat.use_nodes = True
    bsdf = mat.node_tree.nodes.get('Principled BSDF')
    texImage = mat.node_tree.nodes.new('ShaderNodeTexImage')

    # 텍스쳐 이미지 연결
    texImage.image = img
    mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])

    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = mat
    else:
        obj.data.materials.append(mat)


In [83]:
def apply_texture(obj, texture_path):
    # 오브젝트 모드로 전환
    # bpy.ops.object.mode_set(mode='OBJECT')

    # UV 맵 생성
    # bpy.ops.object.mode_set(mode='EDIT')
    # bpy.ops.mesh.select_all(action='SELECT')
    # bpy.ops.uv.smart_project()
    # bpy.ops.object.mode_set(mode='OBJECT')

    # 텍스쳐 이미지 로드
    img = bpy.data.images.load(texture_path)

    # 새 마테리얼 생성
    mat = bpy.data.materials.new(name="CustomMaterial")
    mat.use_nodes = True
    bsdf = mat.node_tree.nodes.get('Principled BSDF')
    texImage = mat.node_tree.nodes.new('ShaderNodeTexImage')

    # 텍스쳐 이미지 연결
    texImage.image = img
    mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])

    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = mat
    else:
        obj.data.materials.append(mat)


def triangulate_object(obj):
    bpy.context.view_layer.objects.active = obj
    bpy.ops.object.mode_set(mode='EDIT')      # 편집 모드로 전환
    bpy.ops.mesh.select_all(action='SELECT')  # 모든 면 선택
    bpy.ops.mesh.quads_convert_to_tris()      # 사각형을 삼각형으로 변환
    bpy.ops.object.mode_set(mode='OBJECT')    # 오브젝트 모드로 다시 전환
    
    
        

def blend_to_fbx(blend_file, fbx_dir, size_multiplier=1, desired_height=1):
    texture_path = 'D:/workspace/Final_Project/AR-zipp/KakaoTalk_20231112_170922211.jpg'
    
    os.makedirs(fbx_dir, exist_ok=True)
    name, extension = blend_file.split("/")[-1].split(".")

    bpy.ops.wm.open_mainfile(filepath=blend_file)

    # Deselect all
    bpy.ops.object.select_all(action='DESELECT')

    # Mesh objects
    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']

    for OBJS in MSH_OBJS[:]:
        # Select all mesh objects
        print("Mesh Object Name:", OBJS.name)
        OBJS.select_set(state=True)
        bpy.context.view_layer.objects.active = OBJS
        
        apply_texture(OBJS, texture_path)

    # Joins objects
    bpy.ops.object.join()

    # Get the current active object
    obj = bpy.context.object

    # Reset the scale
    obj.scale = (1, 1, 1)

    # Apply the scale
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    
    # Apply new dimensions
    new_dimensions = [obj.dimensions.x*size_multiplier, obj.dimensions.y*size_multiplier, desired_height]
    obj.dimensions = new_dimensions
    
    # Auto UV Mapping
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='SELECT')
    bpy.ops.uv.smart_project()
    bpy.ops.object.mode_set(mode='OBJECT')

    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']
    print("")
    print(MSH_OBJS)
    # Apply texture
    apply_texture(obj, texture_path)
    
    # obj = bpy.data.objects.get("Room8")
    # if obj:
    #     print('----------')
    triangulate_object(obj)

    fbx_file = os.path.join(fbx_dir, f'{name}_s{size_multiplier}_h{desired_height}.fbx')

    with contextlib.redirect_stdout(io.StringIO()):
        # bpy.ops.export_scene.fbx(filepath=fbx_file)
        bpy.ops.export_scene.fbx(
                                    filepath=fbx_file,                    # 내보낼 FBX 파일의 경로
                                    axis_forward='-Z',                    # FBX 파일의 전방축 설정
                                    axis_up='Y',                          # FBX 파일의 상방축 설정
                                    use_selection=False,                  # 선택된 오브젝트만 내보낼지 여부
                                    global_scale=1.0,                     # 전역 스케일 팩터
                                    apply_unit_scale=True,                # 단위 스케일 적용
                                    bake_space_transform=False,           # 위치, 회전, 스케일 변환 베이크
                                    object_types={'MESH', 'ARMATURE'},    # 내보낼 오브젝트 유형
                                    use_mesh_modifiers=True,              # 메쉬 수정자 적용
                                    mesh_smooth_type='FACE',               # 메쉬 스무딩 타입
                                    path_mode='COPY',                     # 파일 경로를 복사 모드로 설정
                                    embed_textures=True                   # 사용된 텍스쳐를 FBX 파일에 포함
                                )

    bpy.ops.wm.quit_blender()

    return fbx_file


In [84]:
blend_path = f"statics/blend_file/dfdfdfd.blend"
fbx_dir = 'fbx_test'

fbx_file_path = blend_to_fbx(blend_path, fbx_dir, size_multiplier=2, desired_height=2.5)

Mesh Object Name: Floorplan7
Mesh Object Name: Walls
Mesh Object Name: Box0Wall0
Mesh Object Name: Box0Wall1
Mesh Object Name: Box0Wall2
Mesh Object Name: Box0Wall3
Mesh Object Name: Box0Wall4
Mesh Object Name: Box0Wall5
Mesh Object Name: Box0Wall6
Mesh Object Name: Box0Wall7
Mesh Object Name: Box0Wall8
Mesh Object Name: Box0Wall9
Mesh Object Name: Box0Wall10
Mesh Object Name: Box0Wall11
Mesh Object Name: Box0Wall12
Mesh Object Name: Box0Wall13
Mesh Object Name: Box0Wall14
Mesh Object Name: Box0Wall15
Mesh Object Name: Box0Wall16
Mesh Object Name: Box0Wall17
Mesh Object Name: Box0Wall18
Mesh Object Name: Box0Wall19
Mesh Object Name: Box1Wall20
Mesh Object Name: Box1Wall21
Mesh Object Name: Box1Wall22
Mesh Object Name: Box1Wall23
Mesh Object Name: Box1Wall24
Mesh Object Name: Box1Wall25
Mesh Object Name: Box1Wall26
Mesh Object Name: Box1Wall27
Mesh Object Name: Box1Wall28
Mesh Object Name: Box1Wall29
Mesh Object Name: Box1Wall30
Mesh Object Name: Box1Wall31
Mesh Object Name: Box1Wall32


In [91]:
def create_shared_texture_material(texture_path, material_name="SharedMaterial"):
    # 텍스쳐 이미지 로드
    img = bpy.data.images.load(texture_path)

    # 새 마테리얼 생성
    mat = bpy.data.materials.new(name=material_name)
    mat.use_nodes = True
    bsdf = mat.node_tree.nodes.get('Principled BSDF')
    texImage = mat.node_tree.nodes.new('ShaderNodeTexImage')

    # 텍스쳐 이미지 연결
    texImage.image = img
    mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])

    return mat

def apply_shared_material(obj, shared_material):
    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = shared_material
    else:
        obj.data.materials.append(shared_material)

def blend_to_fbx(blend_file, fbx_dir, size_multiplier=1, desired_height=1):

    os.makedirs(fbx_dir, exist_ok=True)
    name, extension = blend_file.split("/")[-1].split(".")

    bpy.ops.wm.open_mainfile(filepath=blend_file)
    bpy.ops.object.select_all(action='DESELECT')

    texture_path = 'D:/workspace/Final_Project/AR-zipp/KakaoTalk_20231112_170922211.jpg'
    shared_material = create_shared_texture_material(texture_path)
    # Mesh objects
    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']

    for OBJS in MSH_OBJS:
        OBJS.select_set(state=True)
        bpy.context.view_layer.objects.active = OBJS

        # 공유된 머터리얼 적용
        apply_shared_material(OBJS, shared_material)

    # Joins objects
    bpy.ops.object.join()

    # Get the current active object
    obj = bpy.context.object

    # Reset the scale
    obj.scale = (1, 1, 1)

    # Apply the scale
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    
    # Apply new dimensions
    new_dimensions = [obj.dimensions.x*size_multiplier, obj.dimensions.y*size_multiplier, desired_height]
    obj.dimensions = new_dimensions
    
    # Auto UV Mapping
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='SELECT')
    bpy.ops.uv.smart_project()
    bpy.ops.object.mode_set(mode='OBJECT')

    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']
    print("")
    print(MSH_OBJS)
    # Apply texture
    # apply_texture(obj, texture_path)
    
    # obj = bpy.data.objects.get("Room8")
    # if obj:
    #     print('----------')
    triangulate_object(obj)

    fbx_file = os.path.join(fbx_dir, f'{name}_s{size_multiplier}_h{desired_height}.fbx')

    with contextlib.redirect_stdout(io.StringIO()):
        # bpy.ops.export_scene.fbx(filepath=fbx_file)
        bpy.ops.export_scene.fbx(
                                    filepath=fbx_file,                    # 내보낼 FBX 파일의 경로
                                    axis_forward='-Z',                    # FBX 파일의 전방축 설정
                                    axis_up='Y',                          # FBX 파일의 상방축 설정
                                    use_selection=False,                  # 선택된 오브젝트만 내보낼지 여부
                                    global_scale=1.0,                     # 전역 스케일 팩터
                                    apply_unit_scale=True,                # 단위 스케일 적용
                                    bake_space_transform=False,           # 위치, 회전, 스케일 변환 베이크
                                    object_types={'MESH', 'ARMATURE'},    # 내보낼 오브젝트 유형
                                    use_mesh_modifiers=True,              # 메쉬 수정자 적용
                                    mesh_smooth_type='FACE',               # 메쉬 스무딩 타입
                                    path_mode='COPY',                     # 파일 경로를 복사 모드로 설정
                                    embed_textures=True                   # 사용된 텍스쳐를 FBX 파일에 포함
                                )

    bpy.ops.wm.quit_blender()

    return fbx_file


In [92]:
blend_path = f"statics/blend_file/dfdfdfd.blend"
fbx_dir = 'fbx_test'

fbx_file_path = blend_to_fbx(blend_path, fbx_dir, size_multiplier=2, desired_height=2.5)

AttributeError: 'ShaderNodeMapping' object has no attribute 'scale'

In [125]:
def create_shared_texture_material(texture_path, material_name="SharedMaterial", tiling_factor=(2, 2)):
    # 텍스쳐 이미지 로드
    img = bpy.data.images.load(texture_path)

    # 새 마테리얼 생성 및 노드 설정
    mat = bpy.data.materials.new(name=material_name)
    mat.use_nodes = True
    nodes = mat.node_tree.nodes

    # 텍스쳐 노드 생성
    texImage = nodes.new('ShaderNodeTexImage')
    texImage.image = img

    # 맵핑 노드 생성
    mapping = nodes.new('ShaderNodeMapping')
    mapping.vector_type = 'TEXTURE'
    
    # 타일링 스케일 설정
    mapping.inputs['Scale'].default_value[0] = tiling_factor[0]  # X축 타일링
    mapping.inputs['Scale'].default_value[1] = tiling_factor[1]  # Y축 타일링
    mapping.inputs['Scale'].default_value[2] = 1  # Z축은 기본값 유지

    # 텍스쳐 좌표 노드 생성
    texCoord = nodes.new('ShaderNodeTexCoord')

    # 노드 연결
    links = mat.node_tree.links
    links.new(texCoord.outputs['UV'], mapping.inputs['Vector'])
    links.new(mapping.outputs['Vector'], texImage.inputs['Vector'])

    bsdf = nodes.get('Principled BSDF')
    links.new(texImage.outputs['Color'], bsdf.inputs['Base Color'])

    return mat


def apply_shared_material(obj, shared_material):
    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = shared_material
    else:
        obj.data.materials.append(shared_material)

def blend_to_fbx(blend_file, fbx_dir, size_multiplier=1, desired_height=1):

    os.makedirs(fbx_dir, exist_ok=True)
    name, extension = blend_file.split("/")[-1].split(".")

    bpy.ops.wm.open_mainfile(filepath=blend_file)
    bpy.ops.object.select_all(action='DESELECT')

    texture_path = 'D:/workspace/Final_Project/AR-zipp/KakaoTalk_20231112_170922211.jpg'
    shared_material = create_shared_texture_material(texture_path, tiling_factor=(0.1, 0.1))
    # Mesh objects
    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']

    for OBJS in MSH_OBJS:
        OBJS.select_set(state=True)
        bpy.context.view_layer.objects.active = OBJS

        if 'Wall' in OBJS.name and 'Box' in OBJS.name:
            print(OBJS.name)
            # 공유된 머터리얼 적용
            apply_shared_material(OBJS, shared_material)

    # Joins objects
    bpy.ops.object.join()

    # Get the current active object
    obj = bpy.context.object

    # Reset the scale
    obj.scale = (1, 1, 1)

    # Apply the scale
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    
    # Apply new dimensions
    new_dimensions = [obj.dimensions.x*size_multiplier, obj.dimensions.y*size_multiplier, desired_height]
    obj.dimensions = new_dimensions
    
    # Auto UV Mapping
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='SELECT')
    bpy.ops.uv.smart_project()
    bpy.ops.object.mode_set(mode='OBJECT')

    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']
    print("")
    print(MSH_OBJS)
    # Apply texture
    # apply_texture(obj, texture_path)
    
    # obj = bpy.data.objects.get("Room8")
    # if obj:
    #     print('----------')
    triangulate_object(obj)

    fbx_file = os.path.join(fbx_dir, f'{name}_s{size_multiplier}_h{desired_height}.fbx')

    with contextlib.redirect_stdout(io.StringIO()):
        # bpy.ops.export_scene.fbx(filepath=fbx_file)
        bpy.ops.export_scene.fbx(
                                    filepath=fbx_file,                    # 내보낼 FBX 파일의 경로
                                    axis_forward='-Z',                    # FBX 파일의 전방축 설정
                                    axis_up='Y',                          # FBX 파일의 상방축 설정
                                    use_selection=False,                  # 선택된 오브젝트만 내보낼지 여부
                                    global_scale=1.0,                     # 전역 스케일 팩터
                                    apply_unit_scale=True,                # 단위 스케일 적용
                                    bake_space_transform=False,           # 위치, 회전, 스케일 변환 베이크
                                    object_types={'MESH', 'ARMATURE'},    # 내보낼 오브젝트 유형
                                    use_mesh_modifiers=True,              # 메쉬 수정자 적용
                                    mesh_smooth_type='FACE',               # 메쉬 스무딩 타입
                                    path_mode='COPY',                     # 파일 경로를 복사 모드로 설정
                                    embed_textures=True                   # 사용된 텍스쳐를 FBX 파일에 포함
                                )

    bpy.ops.wm.quit_blender()

    return fbx_file


In [126]:
blend_path = f"statics/blend_file/dfdfdfd.blend"
fbx_dir = 'fbx_test'

fbx_file_path = blend_to_fbx(blend_path, fbx_dir, size_multiplier=2, desired_height=2.5)

Box0Wall0
Box0Wall1
Box0Wall2
Box0Wall3
Box0Wall4
Box0Wall5
Box0Wall6
Box0Wall7
Box0Wall8
Box0Wall9
Box0Wall10
Box0Wall11
Box0Wall12
Box0Wall13
Box0Wall14
Box0Wall15
Box0Wall16
Box0Wall17
Box0Wall18
Box0Wall19
Box1Wall20
Box1Wall21
Box1Wall22
Box1Wall23
Box1Wall24
Box1Wall25
Box1Wall26
Box1Wall27
Box1Wall28
Box1Wall29
Box1Wall30
Box1Wall31
Box1Wall32
Box1Wall33
Box2Wall34
Box2Wall35
Box2Wall36
Box2Wall37
Box2Wall38
Box2Wall39
Box2Wall40
Box2Wall41
Box2Wall42
Box2Wall43
Box3Wall44
Box3Wall45
Box3Wall46
Box3Wall47
Box3Wall48
Box4Wall49
Box4Wall50
Box4Wall51
Box4Wall52
Box4Wall53
Box4Wall54
Box4Wall55
Box4Wall56
Box4Wall57
Box4Wall58
Box4Wall59
Box4Wall60
Box4Wall61
Box4Wall62
Box4Wall63
Box4Wall64
Box4Wall65
Box4Wall66
Box4Wall67
Box4Wall68
Box4Wall69
Box4Wall70
Box4Wall71
Box4Wall72
Box4Wall73
Box4Wall74
Box4Wall75
Box4Wall76
Box4Wall77
Box4Wall78
Box4Wall79
Box4Wall80
Box4Wall81
Box4Wall82
Box4Wall83
Box4Wall84
Box4Wall85
Box4Wall86
Box4Wall87
Box4Wall88
Box4Wall89
Box4Wall90
Box4Wall9

In [ ]:
def create_shared_texture_material(texture_path, material_name="SharedMaterial", tiling_factor=(2, 2)):
    # 텍스쳐 이미지 로드
    img = bpy.data.images.load(texture_path)

    # 새 마테리얼 생성 및 노드 설정
    mat = bpy.data.materials.new(name=material_name)
    mat.use_nodes = True
    nodes = mat.node_tree.nodes

    # 텍스쳐 노드 생성
    texImage = nodes.new('ShaderNodeTexImage')
    texImage.image = img

    # 맵핑 노드 생성
    mapping = nodes.new('ShaderNodeMapping')
    mapping.vector_type = 'TEXTURE'
    
    # 타일링 스케일 설정
    mapping.inputs['Scale'].default_value[0] = tiling_factor[0]  # X축 타일링
    mapping.inputs['Scale'].default_value[1] = tiling_factor[1]  # Y축 타일링
    mapping.inputs['Scale'].default_value[2] = 1  # Z축은 기본값 유지

    # 텍스쳐 좌표 노드 생성
    texCoord = nodes.new('ShaderNodeTexCoord')

    # 노드 연결
    links = mat.node_tree.links
    links.new(texCoord.outputs['UV'], mapping.inputs['Vector'])
    links.new(mapping.outputs['Vector'], texImage.inputs['Vector'])

    bsdf = nodes.get('Principled BSDF')
    links.new(texImage.outputs['Color'], bsdf.inputs['Base Color'])

    return mat


def apply_shared_material(obj, shared_material):
    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = shared_material
    else:
        obj.data.materials.append(shared_material)

def blend_to_fbx(blend_file, fbx_dir, size_multiplier=1, desired_height=1):

    os.makedirs(fbx_dir, exist_ok=True)
    name, extension = blend_file.split("/")[-1].split(".")

    bpy.ops.wm.open_mainfile(filepath=blend_file)
    bpy.ops.object.select_all(action='DESELECT')

    texture_path = 'D:/workspace/Final_Project/AR-zipp/KakaoTalk_20231112_170922211.jpg'
    shared_material = create_shared_texture_material(texture_path, tiling_factor=(0.1, 0.1))
    # Mesh objects
    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']

    for OBJS in MSH_OBJS:
        OBJS.select_set(state=True)
        bpy.context.view_layer.objects.active = OBJS

        if 'Wall' in OBJS.name and 'Box' in OBJS.name:
            print(OBJS.name)
            # 공유된 머터리얼 적용
            apply_shared_material(OBJS, shared_material)

    # Joins objects
    bpy.ops.object.join()

    # Get the current active object
    obj = bpy.context.object

    # Reset the scale
    obj.scale = (1, 1, 1)

    # Apply the scale
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    
    # Apply new dimensions
    new_dimensions = [obj.dimensions.x*size_multiplier, obj.dimensions.y*size_multiplier, desired_height]
    obj.dimensions = new_dimensions
    
    # Auto UV Mapping
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='SELECT')
    bpy.ops.uv.smart_project()
    bpy.ops.object.mode_set(mode='OBJECT')

    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']
    print("")
    print(MSH_OBJS)
    # Apply texture
    # apply_texture(obj, texture_path)
    
    # obj = bpy.data.objects.get("Room8")
    # if obj:
    #     print('----------')
    triangulate_object(obj)

    fbx_file = os.path.join(fbx_dir, f'{name}_s{size_multiplier}_h{desired_height}.fbx')

    with contextlib.redirect_stdout(io.StringIO()):
        # bpy.ops.export_scene.fbx(filepath=fbx_file)
        bpy.ops.export_scene.fbx(
                                    filepath=fbx_file,                    # 내보낼 FBX 파일의 경로
                                    axis_forward='-Z',                    # FBX 파일의 전방축 설정
                                    axis_up='Y',                          # FBX 파일의 상방축 설정
                                    use_selection=False,                  # 선택된 오브젝트만 내보낼지 여부
                                    global_scale=1.0,                     # 전역 스케일 팩터
                                    apply_unit_scale=True,                # 단위 스케일 적용
                                    bake_space_transform=False,           # 위치, 회전, 스케일 변환 베이크
                                    object_types={'MESH', 'ARMATURE'},    # 내보낼 오브젝트 유형
                                    use_mesh_modifiers=True,              # 메쉬 수정자 적용
                                    mesh_smooth_type='FACE',               # 메쉬 스무딩 타입
                                    path_mode='COPY',                     # 파일 경로를 복사 모드로 설정
                                    embed_textures=True                   # 사용된 텍스쳐를 FBX 파일에 포함
                                )

    bpy.ops.wm.quit_blender()

    return fbx_file


In [ ]:
blend_path = f"statics/blend_file/dfdfdfd.blend"
fbx_dir = 'fbx_test'

fbx_file_path = blend_to_fbx(blend_path, fbx_dir, size_multiplier=2, desired_height=2.5)

In [137]:
def create_detailed_material(texture_paths, material_name="DetailedMaterial", tiling_factor=(2, 2)):
    # 새 마테리얼 생성 및 노드 설정
    mat = bpy.data.materials.new(name=material_name)
    mat.use_nodes = True
    nodes = mat.node_tree.nodes

    bsdf = nodes.get('Principled BSDF')

    for texture_type, path in texture_paths.items():
        # 텍스처 이미지 로드 및 텍스처 노드 생성
        tex_image = nodes.new('ShaderNodeTexImage')
        tex_image.image = bpy.data.images.load(path)

        # 맵핑 노드 생성 및 설정
        mapping = nodes.new('ShaderNodeMapping')
        mapping.vector_type = 'TEXTURE'
        mapping.inputs['Scale'].default_value = (tiling_factor[0], tiling_factor[1], 1)

        # 텍스쳐 좌표 노드 생성
        texCoord = nodes.new('ShaderNodeTexCoord')

        # 노드 연결
        links = mat.node_tree.links
        links.new(texCoord.outputs['UV'], mapping.inputs['Vector'])
        links.new(mapping.outputs['Vector'], tex_image.inputs['Vector'])

        # 텍스처 타입에 따라 적절한 BSDF 입력에 연결
        if texture_type == 'albedo':
            links.new(tex_image.outputs['Color'], bsdf.inputs['Base Color'])
        elif texture_type == 'ao':
            # AO는 별도의 처리가 필요할 수 있습니다
            pass
        elif texture_type == 'displacement':
            # Displacement는 Material Output 노드에 연결
            material_output = nodes.get('Material Output')
            disp_node = nodes.new('ShaderNodeDisplacement')
            links.new(tex_image.outputs['Color'], disp_node.inputs['Height'])
            links.new(disp_node.outputs['Displacement'], material_output.inputs['Displacement'])
        elif texture_type == 'normal':
            normal_map = nodes.new('ShaderNodeNormalMap')
            links.new(tex_image.outputs['Color'], normal_map.inputs['Color'])
            links.new(normal_map.outputs['Normal'], bsdf.inputs['Normal'])
        elif texture_type == 'roughness':
            links.new(tex_image.outputs['Color'], bsdf.inputs['Roughness'])

    return mat



def apply_shared_material(obj, shared_material):
    # 오브젝트에 마테리얼 할당
    if obj.data.materials:
        obj.data.materials[0] = shared_material
    else:
        obj.data.materials.append(shared_material)


def blend_to_fbx(blend_file, fbx_dir, size_multiplier=1, desired_height=1):

    os.makedirs(fbx_dir, exist_ok=True)
    name, extension = blend_file.split("/")[-1].split(".")

    bpy.ops.wm.open_mainfile(filepath=blend_file)
    bpy.ops.object.select_all(action='DESELECT')

    # 텍스처 경로 정의
    texture_paths = {
                    'albedo': 'D:/workspace/Final_Project/AR-zipp/texture/plaster_damaged_xevjddns/xevjddns_8K_Albedo.jpg',
                    'ao': 'D:/workspace/Final_Project/AR-zipp/texture/plaster_damaged_xevjddns/xevjddns_8K_AO.jpg',
                    'displacement': 'D:/workspace/Final_Project/AR-zipp/texture/plaster_damaged_xevjddns/xevjddns_8K_Displacement.jpg',
                    'normal': 'D:/workspace/Final_Project/AR-zipp/texture/plaster_damaged_xevjddns/xevjddns_8K_Normal.jpg',
                    'roughness': 'D:/workspace/Final_Project/AR-zipp/texture/plaster_damaged_xevjddns/xevjddns_8K_Roughness.jpg'
                    }
    detailed_material = create_detailed_material(texture_paths, tiling_factor=(0.1, 0.1))
    
    # Mesh objects
    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']

    for OBJS in MSH_OBJS:
        OBJS.select_set(state=True)
        bpy.context.view_layer.objects.active = OBJS

        if 'Wall' in OBJS.name and 'Box' in OBJS.name:
            print(OBJS.name)
            # 공유된 머터리얼 적용
            apply_shared_material(OBJS, detailed_material)

    # Joins objects
    bpy.ops.object.join()

    # Get the current active object
    obj = bpy.context.object

    # Reset the scale
    obj.scale = (1, 1, 1)

    # Apply the scale
    bpy.ops.object.transform_apply(location=False, rotation=False, scale=True)
    
    # Apply new dimensions
    new_dimensions = [obj.dimensions.x*size_multiplier, obj.dimensions.y*size_multiplier, desired_height]
    obj.dimensions = new_dimensions
    
    # Auto UV Mapping
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='SELECT')
    bpy.ops.uv.smart_project()
    bpy.ops.object.mode_set(mode='OBJECT')

    MSH_OBJS = [m for m in bpy.context.scene.objects if m.type == 'MESH']
    print("")
    print(MSH_OBJS)
    # Apply texture
    # apply_texture(obj, texture_path)
    
    # obj = bpy.data.objects.get("Room8")
    # if obj:
    #     print('----------')
    triangulate_object(obj)

    fbx_file = os.path.join(fbx_dir, f'{name}_s{size_multiplier}_h{desired_height}.fbx')

    with contextlib.redirect_stdout(io.StringIO()):
        # bpy.ops.export_scene.fbx(filepath=fbx_file)
        bpy.ops.export_scene.fbx(
                                    filepath=fbx_file,                    # 내보낼 FBX 파일의 경로
                                    axis_forward='-Z',                    # FBX 파일의 전방축 설정
                                    axis_up='Y',                          # FBX 파일의 상방축 설정
                                    use_selection=False,                  # 선택된 오브젝트만 내보낼지 여부
                                    global_scale=1.0,                     # 전역 스케일 팩터
                                    apply_unit_scale=True,                # 단위 스케일 적용
                                    bake_space_transform=False,           # 위치, 회전, 스케일 변환 베이크
                                    object_types={'MESH', 'ARMATURE'},    # 내보낼 오브젝트 유형
                                    use_mesh_modifiers=True,              # 메쉬 수정자 적용
                                    mesh_smooth_type='FACE',               # 메쉬 스무딩 타입
                                    path_mode='COPY',                     # 파일 경로를 복사 모드로 설정
                                    embed_textures=True                   # 사용된 텍스쳐를 FBX 파일에 포함
                                )

    bpy.ops.wm.quit_blender()

    return fbx_file


In [138]:
blend_path = f"statics/blend_file/dfdfdfd.blend"
fbx_dir = 'fbx_test'

fbx_file_path = blend_to_fbx(blend_path, fbx_dir, size_multiplier=2, desired_height=2.5)

Box0Wall0
Box0Wall1
Box0Wall2
Box0Wall3
Box0Wall4
Box0Wall5
Box0Wall6
Box0Wall7
Box0Wall8
Box0Wall9
Box0Wall10
Box0Wall11
Box0Wall12
Box0Wall13
Box0Wall14
Box0Wall15
Box0Wall16
Box0Wall17
Box0Wall18
Box0Wall19
Box1Wall20
Box1Wall21
Box1Wall22
Box1Wall23
Box1Wall24
Box1Wall25
Box1Wall26
Box1Wall27
Box1Wall28
Box1Wall29
Box1Wall30
Box1Wall31
Box1Wall32
Box1Wall33
Box2Wall34
Box2Wall35
Box2Wall36
Box2Wall37
Box2Wall38
Box2Wall39
Box2Wall40
Box2Wall41
Box2Wall42
Box2Wall43
Box3Wall44
Box3Wall45
Box3Wall46
Box3Wall47
Box3Wall48
Box4Wall49
Box4Wall50
Box4Wall51
Box4Wall52
Box4Wall53
Box4Wall54
Box4Wall55
Box4Wall56
Box4Wall57
Box4Wall58
Box4Wall59
Box4Wall60
Box4Wall61
Box4Wall62
Box4Wall63
Box4Wall64
Box4Wall65
Box4Wall66
Box4Wall67
Box4Wall68
Box4Wall69
Box4Wall70
Box4Wall71
Box4Wall72
Box4Wall73
Box4Wall74
Box4Wall75
Box4Wall76
Box4Wall77
Box4Wall78
Box4Wall79
Box4Wall80
Box4Wall81
Box4Wall82
Box4Wall83
Box4Wall84
Box4Wall85
Box4Wall86
Box4Wall87
Box4Wall88
Box4Wall89
Box4Wall90
Box4Wall9

In [4]:
import os
 
file_path = 'C:/Users/Sim/Desktop/test/c1'
 
name, extension = os.path.splitext(file_path)
 
# print("확장자: ", os.path.splitext(file_path)[1])
extension

''